In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import skimage as ski

In [ ]:
img0 = ski.io.imread('webreg_0.jpg')
img1 = ski.io.imread('webreg_1.jpg')

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2,
                             figsize=(9, 5))
ax0.imshow(img0)
ax1.imshow(img1)
fig.suptitle('Click 4 matching coordinates; first left image, then right')

coords = []

def onclick(event):
    coords.append((event.xdata, event.ydata))
    
    pcoords = np.array(coords)
    
    ax0.plot(pcoords[::2, 0], pcoords[::2, 1], 'rx')
    ax1.plot(pcoords[1::2, 0], pcoords[1::2, 1], 'wx')
    
callback = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
coords = np.array(coords)
source = coords[::2]
target = coords[1::2]

In [ ]:
tf = ski.transform.estimate_transform('affine', source, target)

Our goal now is to warp the second image and to blend it with the first.

To better display the result (i.e. so pieces of the warped image don't get cropped), we will position both images on a 600x600 canvas.

We use a translation transform to position the images on the canvas.

In [ ]:
offset = ski.transform.AffineTransform(translation=(-200, -170))

img0_warped = ski.transform.warp(img0, inverse_map=offset,
                                 output_shape=(600, 600))

img1_warped = ski.transform.warp(img1, inverse_map=offset + tf,
                                 output_shape=(600, 600))

In [ ]:
# Find where both images overlap; in that region average their values
mask = (img0_warped != 0) & (img1_warped != 0)
registered = img0_warped + img1_warped
registered[mask] /= 2

# Display the results
f, (ax0) = plt.subplots(1, 1, subplot_kw={'xticks': [], 'yticks': []}, figsize=(9, 9))
ax0.imshow(registered);